In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

df_train = pd.read_csv('train_final.csv')
df_test = pd.read_csv('test_final.csv')

In [2]:
target = df_train["Y"]
id = df_test["Id"].values

df_train.drop(['Id','Y'],axis=1,inplace=True)
df_test.drop(['Id'],axis=1,inplace=True)

In [21]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

para = { 'learning_rate': 0.03, 'n_estimators':400, 'max_depth': 4, "random_state":35 ,"verbose":0, "border_count":70, "boosting_type":"Ordered", "class_weights":[0.55,0.45]}

catboost = CatBoostClassifier(**para)
#catboost =  CatBoostClassifier(depth=4,iterations=500,logging_level='Silent',subsample=0.8,bootstrap_type='MVS',eval_metric='AUC')

catboost.fit(df_train, target)

y_proba = catboost.predict_proba(df_train)
print(roc_auc_score(target, y_proba[:,1]))


y_pred = catboost.predict(df_train)

score = (accuracy_score(target, y_pred)) * 100
print("Accuracy score is %.2f%%" % score)

score = cross_val_score(catboost, df_train, target, scoring="roc_auc", cv=5)
print ("Cross validation score for catboost: ",np.mean(score)*100)

0.9468901278840219
Accuracy score is 87.98%
Cross validation score for catboost:  90.13474653536677


In [ ]:
0.9468901278840219
Accuracy score is 87.98%
Cross validation score for catboost:  90.13474653536677

In [17]:
sub_proba = catboost.predict_proba(df_test)
pred_col=np.c_[id,sub_proba[:,1]]

submission = pd.DataFrame(pred_col, columns = ['Id','Y'])
submission["Id"] = submission["Id"].astype("Int32")
submission

,Id,Y
0,2604,0.060650
1,2605,0.071617
2,2606,0.021078
3,2607,0.014043
4,2608,0.782579
...,...,...
3599,6203,0.171742
3600,6204,0.027070
3601,6205,0.005372
3602,6206,0.050801


In [18]:
submission.to_csv('post1submission.csv', index=False)